<a href="https://colab.research.google.com/github/rswijesena/AI/blob/main/PinecodeVectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector DB

In [ ]:
#!pip install langchain
#!pip install pinecone-client
#!pip install openai
#!pip install pypdf
#!pip install tiktoken
!pip install langchain-community

In [59]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 5.6 MB/s eta 0:00:00


In [60]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os


In [6]:
!mkdir pdfs

In [10]:
loader = PyPDFDirectoryLoader("pdfs")

In [11]:
data = loader.load()

In [ ]:
data[0]

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [15]:
text_chunks = text_splitter.split_documents(data)

In [43]:
len(text_chunks[1].page_content)

431

In [68]:
import os
os.environ["OPENAI_API_KEY"] = "---"

os.environ["PINECONE_API_KEY"] = "---"

In [31]:
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings.embed_query("How are you?")

In [70]:
from langchain_pinecone import PineconeVectorStore

In [73]:
index_name = "firstvector"
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)


In [92]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x7c9596b62f80>)

In [89]:
query = "what is Scaled Dot-Product Attention?"

In [90]:
docs = docsearch.similarity_search(query)

In [83]:
docs[0]

Document(page_content='Decoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two\nsub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head\nattention over the output of the encoder stack. Similar to the encoder, we employ residual connections\naround each of the sub-layers, followed by layer normalization. We also modify the self-attention\nsub-layer in the decoder stack to prevent positions from attending to subsequent positions. This', metadata={'page': 2.0, 'source': 'pdfs/allyouneed.pdf'})

In [86]:
llm = OpenAI(temperature=0)

In [87]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [91]:
qa.run(query)

' Scaled Dot-Product Attention is a type of attention mechanism used in neural networks, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. It is called "scaled" because the dot products are scaled by a factor of 1/sqrt(dk) to prevent extremely large gradients.'

In [97]:
import sys
while True:
  user_input = input(f"Input Prompt:" )
  if user_input == "exit":
    sys.exit()
  if user_input == '':
    continue
  result = qa.run({'query': user_input})
  print(result)

Input Prompt:who is Jimmy Lei Ba?
 I don't know.
Input Prompt:what did who is Jimmy Lei Ba do?
 I don't know.
Input Prompt:what Jimmy Lei Ba done?
 I don't know.
Input Prompt:any acknowledgements?
 Yes, the acknowledgements are listed in the context provided. They include Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, and Aidan N. Gomez.
Input Prompt:is the code available somewhere?
 Yes, the code used to train and evaluate the models is available at https://github.com/tensorflow/tensor2tensor.
Input Prompt:exit


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
